In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import myutils as u
import math
import datetime
import warnings
import pandas.errors
warnings.simplefilter(action='ignore', category=Warning)

# Import Data

In [2]:
ALL_TC_SITES = u.import_data("SITES")
ALL_TC = u.import_data("TRAFFIC_COUNTS")
ALL_OTP = u.import_data("ON_TIME")
ALL_STOPS = u.import_data("STOPS")
ALL_CONSTR = u.import_data("LANE_CLOSURE")
ALL_ROADS = u.import_data("ROAD_NETWORK")

# Consts
MIN = pd.to_timedelta("1 min")
DAY = pd.to_timedelta("1 day")
SITES_NO = {'McPhillips': 0,'Henderson': 1,'Pembina': 2,'Inkster': 3,'Nichol': 4,'Lagimodiere': 5,'Disraeli': 6,'Marion': 7} 

In [23]:
def get_stops_distances(tc_site, distance=500,key=None):
    TC_SITE = ALL_TC_SITES[ALL_TC_SITES["Street"] == tc_site]
    site_coords = TC_SITE[["Lat","Long"]].values[0]
    stop_coords = ALL_STOPS[["Lat","Long"]].values
    distances = u.distance_within(site_coords,stop_coords,distance,key=key)
    return distances 

def get_stops_nearby(tc_site, distance=500):
    stops_index = get_stops_distances(tc_site, distance, "index")
    return ALL_STOPS.iloc[stops_index]

def get_traffic_counts(tc_site,date,freq=None):
    # Obtain the traffic count information
    TC_SITE = ALL_TC_SITES[ALL_TC_SITES["Street"] == tc_site]
    TC = ALL_TC[(ALL_TC["Timestamp"]>date) & (ALL_TC["Timestamp"]<= date+1*DAY) & (ALL_TC["Street"]==TC_SITE["Street"].item())]
    TC.loc[:,"Time Interval"] = [u.fmt_timestamp(i) for i in TC["Timestamp"]]

    if freq != None:
    # Select the frequency with which the data is aggregated
        time_range = date + pd.timedelta_range(start="0:00:00",end="24:00:00",freq=freq)
        AGGR_TC = pd.DataFrame()
        cols = ["Northbound","Southbound","Eastbound","Westbound","Total"]
        for i in range(len(time_range)-1):
            lower_lim = time_range[i]
            upper_lim = time_range[i+1]
            res = TC[TC["Timestamp"] == upper_lim]
            res.loc[:,cols] = TC.loc[(TC["Timestamp"] <= upper_lim) & (TC["Timestamp"] > lower_lim),cols].sum(axis=0).values
            AGGR_TC = pd.concat([AGGR_TC,res])
        TC = AGGR_TC
    return TC.sort_values("Timestamp")

def get_otp(start,end):
    return ALL_OTP[(ALL_OTP["Scheduled Time"] <= end) & (ALL_OTP["Scheduled Time"] > start)]

def prepare_data(tc_site,date,distance=500,freq=None):
    AFF_STOPS = get_stops_nearby(tc_site, distance)
    distances = get_stops_distances(tc_site, distance, key="distance")
    TC = get_traffic_counts(tc_site,date,freq)
    OTP = get_otp(date, date+1*DAY)
    DF = pd.DataFrame()
    for timestamp in TC["Timestamp"]:
        TC_i = TC[TC["Timestamp"] == timestamp]
        df1 = AFF_STOPS.loc[:,["Stop Number"]]
        df1.loc[:,"Site"] = SITES_NO[tc_site]
        df1.loc[:,"Distance"] = distances
        df1.loc[:,"Same Street"] = (AFF_STOPS["Street"] == TC_i["Street"].item()).replace({True:1,False:0})
        # Directional & Total traffic count
        df1.loc[(AFF_STOPS["Street"] != tc_site),"Directional"] = 0
        for direction in ["Northbound","Southbound","Eastbound","Westbound"]:
            df1.loc[(AFF_STOPS["Street"] == tc_site) & (AFF_STOPS["Direction"] == direction),"Directional"] = TC_i[direction].item()
        df1.loc[:,"Total"] = TC_i["Total"].item()
        # 
        df1.loc[:,["Arrivals","Average OTP"]] = [(len(OTP.loc[OTP["Stop Number"]==stop_no,"Deviation"].values),OTP.loc[OTP["Stop Number"]==stop_no,"Deviation"].values.mean()) 
                                                                for stop_no in AFF_STOPS["Stop Number"]]
        df1.loc[:,"Time interval"] = u.fmt_timestamp(timestamp)
        df1.loc[:,"Date"] = TC_i["Timestamp"].item().date()
        DF = pd.concat([DF,df1])
    return DF

date = pd.to_datetime("2021-08-23")
tc_site = "McPhillips"
distance=500
freq = "2h"

AFF_STOPS = get_stops_nearby(tc_site, distance)
TC = get_traffic_counts(tc_site,date,freq)
prepare_data(tc_site,date,distance,freq)
# AFF_STOPS

,Stop Number,Site,Distance,Same Street,Directional,Total,Arrivals,Average OTP,Time interval,Date
1483,30199,0,421.070384,0,0.0,501.0,33,-84.787879,2.0,2021-08-23
1628,30353,0,329.076830,1,281.0,501.0,65,-3.646154,2.0,2021-08-23
1637,30362,0,381.455906,0,0.0,501.0,93,55.172043,2.0,2021-08-23
1638,30363,0,263.378003,0,0.0,501.0,109,-50.266055,2.0,2021-08-23
1639,30364,0,271.147088,0,0.0,501.0,15,-55.133333,2.0,2021-08-23
...,...,...,...,...,...,...,...,...,...,...
2150,30918,0,337.529056,0,0.0,1680.0,14,-22.642857,0.0,2021-08-24
2151,30919,0,342.831977,0,0.0,1680.0,14,-19.571429,0.0,2021-08-24
2157,30925,0,468.780146,0,0.0,1680.0,9,-54.888889,0.0,2021-08-24
2158,30926,0,350.956296,0,0.0,1680.0,9,-54.888889,0.0,2021-08-24


In [4]:
ALL_CONSTR.head()

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",2021-08-16,2021-10-16,00:00:00,00:00:00,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,00:00:00,00:00:00,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,00:00:00,00:00:00,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,00:00:00,23:00:00,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,00:00:00,00:00:00,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2


In [5]:
date = pd.to_datetime("2021-08-23")
distance=500
time = pd.to_datetime("16:00:00").time()

Timedelta('0 days 04:00:00')

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",2021-08-16,2021-10-16,00:00:00,00:00:00,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,00:00:00,00:00:00,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,00:00:00,00:00:00,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,00:00:00,23:00:00,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,00:00:00,00:00:00,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,Munroe,Henderson,"[Henderson Hw, Golspie St]",[Westbound],2021-05-03,2021-10-08,00:00:00,00:00:00,Yes,49.919450,-97.103062,"[[(49.92159361799, -97.109515392943), (49.9214...",1,7
403,Stafford,Dudley,"[Dudley Av, Warsaw Av]",[Northbound],2021-08-17,2021-10-15,00:00:00,00:00:00,No,49.865149,-97.159055,"[[(49.8637858332, -97.157961213665), (49.86386...",1,17
404,Bishop Grandin,Lagimodiere,"[Lagimodiere Bv, De La Seigneurie Bv]",[Eastbound],2021-08-07,2021-10-15,00:00:00,00:00:00,No,49.840370,-97.053552,"[[(49.840625586473, -97.057348732447), (49.840...",1,15
405,Winterton,Roch,"[Roch St, Watt St]",[Southbound],2021-08-19,2021-10-08,00:00:00,00:00:00,No,49.917711,-97.100369,"[[(49.918363079885, -97.102331187677), (49.917...",1,3


In [22]:
a = get_construction_sites(date,time=pd.to_datetime("16:00:00").time()).loc[406,"Geometry"]
from itertools import chain
list(chain.from_iterable(a))
# b = []
# for i in a:
#     b += i
# b

[(49.840279352993, -97.049518603176),
 (49.840896491873, -97.04951995419),
 (49.841444485241, -97.04952115367),
 (49.843003715326, -97.049525971963),
 (49.844490306683, -97.049530263365),
 (49.833864335216, -97.04956685375),
 (49.836684714034, -97.049529984707),
 (49.837782327298, -97.049513134844),
 (49.840110059866, -97.049518232534)]